In [ ]:
# Setup redis
%pip install redis

In [ ]:
fullName = "Full Name" # <--- PUT YOUR FULL NAME HERE (e.g. Jimmy Jones)

assert fullName != "Full Name"

password = input("Enter the password displayed on the projector in class! \nPassword: ")
print(f"You entered: {password}")

In [ ]:
# Connect

import redis
import time

host = 'redis-14806.c1.asia-northeast1-1.gce.redns.redis-cloud.com'
port = 14806   
username = 'CS452'

r = redis.Redis(host=host, port=port, password=password, db=0, username=username)

In [ ]:
# set 

r.set(fullName, 'hello world') # True

In [ ]:
# get
result = r.get(fullName)

print(result.decode())

In [ ]:
# set with expiry!

myKey = 'expiringKey_' + fullName
r.set(myKey, "I only get to be here for a short time", ex=3)

start = time.time()
while result := r.get(myKey):
    print(result, time.time() - start, myKey)

print("result gone!")

In [ ]:
# delete

r.delete(fullName)
ans = r.get(fullName)

if ans == None:
    print(f'key "{fullName}" is not there anymore.')
else:
    print(f'key "{fullName}" is {ans.decode()}') 

In [ ]:
# sets! and sismember

setKey = "iwashere"
print("Did you sign the guest book?")
print("yes" if r.sismember(setKey, fullName) else "no")

In [ ]:
# sadd

r.sadd(setKey, fullName)

In [ ]:
# sismember

print("Did you sign the guest book?")
print("yes" if r.sismember(setKey, fullName) else "no")

In [ ]:
# scard and smembers

print(f"We have {r.scard(setKey)} {'entry' if r.scard(setKey) == 1 else 'entries'}!")
print(r.smembers("iwashere"))

In [ ]:
# incr

print("ring the bell!")
result = r.incr("bell")

print(f"This was the ring number {result}!")

# When would this be useful? Inventory?

In [ ]:
# Unsafe Transfers..

for _ in range(4):
    print("--- LOOP ---")

    # Starting amounts
    print(f'Staring: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')


    # Unsafe transfer $100
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) - 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) + 100)

    print(f'Transfered: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    # Unsafe transfer the money back!
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) + 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) - 100)

    # Ending amounts
    print(f'Transfered back: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

In [ ]:
# Fix Fred and George's accounts
r.set("fredsMoney", 10000)
r.set("georgesMoney", 10000)

In [ ]:
# Transfer with locking!  set(ex and nx = True)

# Safe Transfers..
safeTransferLockName = "cs452PatienceLock"
def getLock():
    print("Waiting for lock...", end="")
    while True:
        if r.set(safeTransferLockName, "MINE!", ex=2, nx=True):
            print()
            print("SET LOCK")
            return
        print(".", end="")
        time.sleep(.01)

def releaseLock():
    print("RELEASED LOCK")
    r.delete(safeTransferLockName)


for _ in range(4):
    print("--- LOOP ---")
    getLock()  # <--------- GET THE LOCK ----------------<<

    # Starting amounts
    print(f'Staring: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')


    # Safe transfer $100
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) - 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) + 100)

    print(f'Transfered: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    # Safe transfer the money back!
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) + 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) - 100)

    # Ending amounts
    print(f'Transfered back: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    releaseLock() # <------------ RELEASE THE LOCK -------------------<<

In [ ]:
# Barriers and work queues
barrierKey = "everyOneIsWaitingForThis"


In [ ]:
# Restore the work queue
r.rpush("serviceQueueKey", f"Wash dishes for {fullName}")
r.rpush("serviceQueueKey", f"Cook dinner for {fullName}")
r.rpush("serviceQueueKey", *[f"Service Opportunity for ({fullName})" for _ in range(3)])

In [ ]:
# Barrier
timeout = time.time() + 120 # two minutes
def waitOnBarrier():
    print("Wait for barrier...")
    while not r.get(barrierKey):
        print(".", end="")
        time.sleep(.01)
        if time.time() > timeout:
            raise TimeoutError("Waiting too long for barrier! Try again?")
    
    print()
    print("Go!")

waitOnBarrier()

# Pull work off of work queue!
for _ in range(10):
    result = r.lpop("serviceQueueKey")
    print(result)

In [ ]:
# Allow Barrier
r.set(barrierKey, "Let them go!", ex=5)

In [ ]:
# Block Barrier  
r.delete(barrierKey) # Should be an expiring key already, but just in case someone put something else in :)